In [ ]:
pip install pygame

In [ ]:
########################################################
# SWITCHING FILTER
########################################################

import pygame
import math
import random
import numpy as np

pygame.init()

########################################################
# Settings
########################################################
EVASION_THRESHOLD = 40
w, h = 800, 600
evader_speed = 8
pursuer_speed = 1
evader_radius, pursuer_radius, goal_radius = 15, 15, 20
max_goals = 10

# Colors
WHITE, BLUE, RED, GREEN, BLACK = (255,255,255), (50,150,255), (255,80,80), (50,200,50), (0,0,0)

# Initial positions
evader_x =  w//4
evader_y =  h//2
pursuer_x = 3*w//4 
pursuer_y = h//2
goal_counter = 0
goal_times = []

#Set up game
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Pursuer-Evader Game")
clock = pygame.time.Clock()

# Initialize first goal
goal_x, goal_y = random.randint(0, w), random.randint(0, h)
goal_spawn_time = pygame.time.get_ticks() / 1000.0
running = True
game_over = False


while running:
    #set the speed of the game
    clock.tick(120)
    screen.fill(WHITE)

    if not game_over:
        ########################################################
        # MOVEEEE
        ########################################################
        #caluculate hte current distance between the pursuer and the evader
        pursuer_dx = pursuer_x - evader_x
        pursuer_dy = pursuer_y - evader_y
        dist_to_pursuer = np.sqrt(pursuer_dx**2+ pursuer_dy**2)

        # engage in straight line movement towards goal or evasion
        if dist_to_pursuer >= EVASION_THRESHOLD:
            # Go to goal via a straight line
            goal_dx = goal_x - evader_x
            goal_dy = goal_y - evader_y
            goal_dist = np.sqrt(goal_dx**2+ goal_dy**2)
            #if the distance is greater than 0, move the evader towards the goal
            if goal_dist > 0:
                evader_x += (goal_dx / goal_dist) * evader_speed
                evader_y += (goal_dy / goal_dist) * evader_speed
        else:
            # Evasion mode: move away from pursuer
            # Calculate direction away from pursuer (evader to pursuer reversed)
            evasion_dx = evader_x - pursuer_x
            evasion_dy = evader_y - pursuer_y
            evasion_dist = np.sqrt(evasion_dx**2 + evasion_dy**2)
            if evasion_dist > 0:
                # Normalize and move away from pursuer
                evader_x += (evasion_dx / evasion_dist) * evader_speed
                evader_y += (evasion_dy / evasion_dist) * evader_speed
       
        # update the position of the evader
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # Pursuer chases evader
        capture_distance_dx = evader_x - pursuer_x
        capture_distance_dy = evader_y - pursuer_y
        capture_distance = np.sqrt(capture_distance_dx**2 + capture_distance_dy**2)
        if capture_distance > 0:
            pursuer_x += (capture_distance_dx / capture_distance) * pursuer_speed
            pursuer_y += (capture_distance_dy / capture_distance) * pursuer_speed

        # update the position of the pursuer
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x))
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))

        ########################################################
        # CHECK COLLISIONS WITH ADVERSARY AND GOAL
        ########################################################
        #current distance between the pursuer and the evader
        capture_distance_dx_postupdate = evader_x - pursuer_x
        capture_distance_dy_postupdate = evader_y - pursuer_y
        capture_distance_post_update = np.sqrt(capture_distance_dx_postupdate**2 + capture_distance_dy_postupdate**2)
        if capture_distance_post_update <= evader_radius + pursuer_radius:
            game_over = True
            running = False

        # Check goal collection
        goal_distance_dx_postupdate = evader_x - goal_x
        goal_distance_dy_postupdate = evader_y - goal_y
        goal_distance_post_update = np.sqrt(goal_distance_dx_postupdate**2 + goal_distance_dy_postupdate**2)
        if goal_counter < max_goals and goal_distance_post_update <= evader_radius + goal_radius:
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time)
            goal_counter += 1
            if goal_counter < max_goals:
                goal_x, goal_y = random.randint(0, w), random.randint(0, h)
                goal_spawn_time = pygame.time.get_ticks() / 1000.0
            else:
                game_over = True
                running = False

    ########################################################
    # Draw circles
    ########################################################
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    #draw the line between the pursuer and the evader for visualization
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)
    if goal_counter < max_goals:
        pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)
    

    # Display
    font = pygame.font.Font(None, 36)
    if not game_over:
        screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 10))
        dist_to_pursuer = math.hypot(pursuer_x - evader_x, pursuer_y - evader_y)
        if goal_counter < max_goals and dist_to_pursuer >= EVASION_THRESHOLD:
            screen.blit(font.render("Mode: GOAL-SEEKING", True, GREEN), (10, 50))
        else:
            screen.blit(font.render("Mode: EVASION", True, RED), (10, 50))
        screen.blit(font.render("C: close", True, BLACK), (10, 90))
    else:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals collected!", True, GREEN)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))
        else:
            msg = font.render("GAME OVER - Caught!", True, RED)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))

    pygame.display.flip()

pygame.quit()

In [ ]:
########################################################
# CBF
########################################################

import pygame
import math
import random
import numpy as np

pygame.init()

########################################################
# Settings
########################################################

# --- CHOOSE FILTER ---
# 'SWITCHING' = Original threshold-based evasion
# 'CBF' = Control Barrier Function
FILTER_MODE = 'CBF' 
# ---------------------

# Switching Filter settings
EVASION_THRESHOLD = 50 # Original was 40, increased for better comparison

# CBF settings
CBF_SAFETY_RADIUS = 80  # Safety radius 'r' (h = d^2 - r^2) - INCREASED from 60
CBF_ALPHA = 0.5         # CBF gain 'alpha' (h_dot >= -alpha * h)
STRAY_RADIUS = 150      # NEW: Radius at which to start "straying" away

# General game settings
w, h = 800, 600
evader_speed = 8
pursuer_speed = 1 # REVERTED from 2 to 1
evader_radius, pursuer_radius, goal_radius = 15, 15, 20 # REVERTED evader_radius from 20 to 15
COLLISION_DISTANCE = evader_radius + pursuer_radius
max_goals = 10

# Colors
WHITE, BLUE, RED, GREEN, BLACK = (255,255,255), (50,150,255), (255,80,80), (50,200,50), (0,0,0)
PURPLE = (150, 0, 150)

# Initial positions
evader_x =  w//4
evader_y =  h//2
pursuer_x = 3*w//4 
pursuer_y = h//2
goal_counter = 0
goal_times = []

#Set up game
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption(f"Pursuer-Evader Game - Mode: {FILTER_MODE}")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)
small_font = pygame.font.Font(None, 28)

# Initialize first goal
goal_x, goal_y = random.randint(0, w), random.randint(0, h)
goal_spawn_time = pygame.time.get_ticks() / 1000.0
running = True
game_over = False

# State variable to show when CBF is active
cbf_is_active = False
# State variable for behavior weighting
w_goal = 1.0
w_evade = 0.0

########################################################
# Control Barrier Function (CBF)
########################################################
def cbf_filter(evader_x, evader_y, pursuer_x, pursuer_y, desired_vx, desired_vy):
    """
    Applies a CBF to the desired velocity to ensure safety.
    Based on the logic from realworld_withCBF.ipynb.
    """
    global cbf_is_active
    
    # Relative position vectors
    rel_x = evader_x - pursuer_x
    rel_y = evader_y - pursuer_y
    
    # Safety margin function h = d^2 - r^2
    # h > 0: Safe
    # h = 0: On boundary
    # h < 0: Unsafe
    h = rel_x**2 + rel_y**2 - CBF_SAFETY_RADIUS**2
    
    # h_dot = 2 * (rel_x * rel_vx + rel_y * rel_vy)
    # We assume pursuer is static for this calculation (rel_v = evader_v)
    # This matches the simplified logic from your notebook.
    h_dot = 2 * (rel_x * desired_vx + rel_y * desired_vy)
    
    # CBF safety condition: h_dot >= -alpha * h
    if h_dot >= -CBF_ALPHA * h:
        # Action is safe, return the desired velocity
        cbf_is_active = False
        return desired_vx, desired_vy

    # --- Action is NOT safe ---
    # The desired velocity violates the CBF condition.
    # Instead of stopping (return 0,0), we will modify the velocity
    # to be "tangent" to the safety circle, implementing the user's
    # "go orthogonal" idea.

    cbf_is_active = True # We know we are intervening

    # Vector pointing from pursuer to evader (normal to the circle)
    # rel_x and rel_y are already defined above
    dist = np.sqrt(rel_x**2 + rel_y**2)
    
    if dist == 0:
        return 0, 0 # On top of pursuer, just stop

    # Normalized normal vector (pointing outwards from pursuer)
    norm_x = rel_x / dist
    norm_y = rel_y / dist
    
    # Project desired velocity onto the normal vector
    # This gives the scalar magnitude of the velocity in the normal direction
    v_normal_scalar = desired_vx * norm_x + desired_vy * norm_y
    
    # v_normal_scalar > 0 means we are moving away (outwards)
    # v_normal_scalar < 0 means we are moving towards (inwards)
    
    # Now, calculate the purely tangential component of the velocity
    # v_tangent = v_desired - v_normal
    v_tangent_x = desired_vx - (v_normal_scalar * norm_x)
    v_tangent_y = desired_vy - (v_normal_scalar * norm_y)
    
    # This new velocity (v_tangent_x, v_tangent_y) is purely
    # orthogonal to the pursuer. Its h_dot will be 0.
    # The CBF condition becomes: 0 >= -CBF_ALPHA * h
    # This will always be true as long as h >= 0 (we are on or outside
    # the barrier), so this move is guaranteed to be safe.
    
    # Return the "sliding" velocity
    return v_tangent_x, v_tangent_y

########################################################
# Main Game Loop
########################################################
while running:
    # Set the speed of the game
    clock.tick(60) # Capped at 60 FPS for stable physics
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                running = False

    if not game_over:
        ########################################################
        # 1. Calculate DESIRED Evader Velocity (Blended)
        ########################################################
        
        # 1a. Get goal-seeking vector
        goal_dx = goal_x - evader_x
        goal_dy = goal_y - evader_y
        goal_dist = np.sqrt(goal_dx**2 + goal_dy**2)
        v_goal_x, v_goal_y = 0, 0
        if goal_dist > 0:
            v_goal_x = (goal_dx / goal_dist) * evader_speed
            v_goal_y = (goal_dy / goal_dist) * evader_speed

        # 1b. Get evasion vector
        pursuer_dx = evader_x - pursuer_x
        pursuer_dy = evader_y - pursuer_y
        dist_to_pursuer = np.sqrt(pursuer_dx**2 + pursuer_dy**2)
        v_evade_x, v_evade_y = 0, 0
        if dist_to_pursuer > 0:
            v_evade_x = (pursuer_dx / dist_to_pursuer) * evader_speed
            v_evade_y = (pursuer_dy / dist_to_pursuer) * evader_speed
        
        # 1c. Calculate weights based on distance
        if dist_to_pursuer > STRAY_RADIUS:
            w_goal = 1.0
            w_evade = 0.0
        elif dist_to_pursuer < CBF_SAFETY_RADIUS:
            # Inside the "hard" safety radius, desire should be 100% evasive
            w_goal = 0.0
            w_evade = 1.0
        else:
            # We are in the "stray zone" (between STRAY_RADIUS and CBF_SAFETY_RADIUS)
            # Linearly interpolate the weights.
            # When dist = STRAY_RADIUS, w_evade = 0
            # When dist = CBF_SAFETY_RADIUS, w_evade = 1
            w_evade = 1.0 - (dist_to_pursuer - CBF_SAFETY_RADIUS) / (STRAY_RADIUS - CBF_SAFETY_RADIUS)
            w_goal = 1.0 - w_evade

        # 1d. Combine velocities
        desired_vx = v_goal_x * w_goal + v_evade_x * w_evade
        desired_vy = v_goal_y * w_goal + v_evade_y * w_evade

        # 1e. Renormalize the final desired velocity to maintain speed
        desired_norm = np.sqrt(desired_vx**2 + desired_vy**2)
        if desired_norm > 0:
            desired_vx = (desired_vx / desired_norm) * evader_speed
            desired_vy = (desired_vy / desired_norm) * evader_speed
        
        ########################################################
        # 2. Apply Safety Filter to get SAFE Velocity
        ########################################################
        safe_vx, safe_vy = 0, 0
        
        if FILTER_MODE == 'SWITCHING':
            # --- Switching Filter Logic ---
            dist_to_pursuer = np.sqrt((pursuer_x - evader_x)**2 + (pursuer_y - evader_y)**2)
            
            if dist_to_pursuer >= EVASION_THRESHOLD:
                # Mode: GOAL-SEEKING
                safe_vx, safe_vy = desired_vx, desired_vy
            else:
                # Mode: EVASION
                evasion_dx = evader_x - pursuer_x
                evasion_dy = evader_y - pursuer_y
                evasion_dist = np.sqrt(evasion_dx**2 + evasion_dy**2)
                if evasion_dist > 0:
                    safe_vx = (evasion_dx / evasion_dist) * evader_speed
                    safe_vy = (evasion_dy / evasion_dist) * evader_speed
                else:
                    safe_vx, safe_vy = 0, 0 # On top of pursuer
            
        elif FILTER_MODE == 'CBF':
            # --- CBF Filter Logic ---
            safe_vx, safe_vy = cbf_filter(evader_x, evader_y, pursuer_x, pursuer_y, desired_vx, desired_vy)

        ########################################################
        # 3. Update Evader Position
        ########################################################
        evader_x += safe_vx
        evader_y += safe_vy
        
        # Keep evader on screen
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        ########################################################
        # 4. Update Pursuer Position
        ########################################################
        capture_distance_dx = evader_x - pursuer_x
        capture_distance_dy = evader_y - pursuer_y
        capture_distance = np.sqrt(capture_distance_dx**2 + capture_distance_dy**2)
        
        if capture_distance > 0:
            pursuer_x += (capture_distance_dx / capture_distance) * pursuer_speed
            pursuer_y += (capture_distance_dy / capture_distance) * pursuer_speed

        # Keep pursuer on screen
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x))
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))

        ########################################################
        # 5. Check Collisions and Goals
        ########################################################
        # Check for capture
        final_dist = np.sqrt((evader_x - pursuer_x)**2 + (evader_y - pursuer_y)**2)
        if final_dist <= COLLISION_DISTANCE:
            game_over = True
            running = False # Or set a game_over flag

        # Check for goal collection
        final_goal_dist = np.sqrt((evader_x - goal_x)**2 + (evader_y - goal_y)**2)
        if goal_counter < max_goals and final_goal_dist <= evader_radius + goal_radius:
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time)
            goal_counter += 1
            if goal_counter < max_goals:
                goal_x, goal_y = random.randint(0, w), random.randint(0, h)
                goal_spawn_time = pygame.time.get_ticks() / 1000.0
            else:
                game_over = True # Victory

    ########################################################
    # Draw Everything
    ########################################################
    
    # Draw safety radii
    if FILTER_MODE == 'SWITCHING':
        # Draw switching threshold
        pygame.draw.circle(screen, (255, 200, 200), (int(pursuer_x), int(pursuer_y)), EVASION_THRESHOLD, 2)
    elif FILTER_MODE == 'CBF':
        # Draw CBF safety radius
        pygame.draw.circle(screen, (200, 200, 255), (int(pursuer_x), int(pursuer_y)), CBF_SAFETY_RADIUS, 2)
        # Draw "stray" radius
        pygame.draw.circle(screen, (220, 220, 220), (int(pursuer_x), int(pursuer_y)), STRAY_RADIUS, 1)

    # Draw agents
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    
    # Draw line between them
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)
    
    # Draw goal
    if goal_counter < max_goals:
        pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)
    
    # Display Text
    screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 10))
    screen.blit(small_font.render(f"Filter Mode: {FILTER_MODE}", True, PURPLE), (10, 50))
    
    if FILTER_MODE == 'SWITCHING':
        dist_to_pursuer = np.sqrt((pursuer_x - evader_x)**2 + (pursuer_y - evader_y)**2)
        if dist_to_pursuer < EVASION_THRESHOLD:
            screen.blit(font.render("Mode: EVASION", True, RED), (10, 80))
        else:
            screen.blit(font.render("Mode: GOAL-SEEKING", True, GREEN), (10, 80))
    
    elif FILTER_MODE == 'CBF':
        if cbf_is_active:
            screen.blit(font.render("CBF ACTIVE: SLIDING", True, RED), (10, 80))
        elif w_evade > 0:
            screen.blit(font.render("Mode: STRAYING", True, (255, 165, 0)), (10, 80))
        else:
            screen.blit(font.render("Mode: GOAL-SEEKING", True, GREEN), (10, 80))
        
        # Show weights
        screen.blit(small_font.render(f"W_Goal: {w_goal:.2f}", True, BLACK), (10, 110))
        screen.blit(small_font.render(f"W_Evade: {w_evade:.2f}", True, BLACK), (10, 135))

    # Game Over Message
    if game_over:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals collected!", True, GREEN)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))
        else:
            msg = font.render("GAME OVER - Caught!", True, RED)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))

    pygame.display.flip()

# Wait 2 seconds on game over before quitting
if game_over:
    pygame.time.wait(2000)

pygame.quit()

In [ ]:
import pygame
import math
import random
import numpy as np

pygame.init()

########################################################
# Settings (CBF-only, two-pursuer with cooperation)
########################################################
w, h = 800, 600
evader_speed = 8.0

# Base pursuer speeds; we grant a coop boost after black swan spawns
PURSUER_SPEED_BASE = 1.2
PURSUER_SPEED_COOP  = 2.5   # active when p2 joins

evader_radius, pursuer_radius, goal_radius = 15, 15, 20
COLLISION_DISTANCE = evader_radius + pursuer_radius
max_goals = 10

# CBF + behavior shaping
CBF_SAFETY_RADIUS = 80.0     # "hard" safety
STRAY_RADIUS      = 150.0    # "soft" outer zone (blend away)
CBF_ALPHA         = 2.0      # conservative
CBF_EPS           = 1e-9
CBF_MAX_ITERS     = 3        # projection rounds for multiple constraints

# Cooperation params
BLOCK_FRACTION         = 0.55       # where the blocker sits along evader->goal (0..1)
BLOCK_STANDOFF         = 40.0       # keep a little stand-off from goal
BLOCK_REPOSITION_GAIN  = 1.0        # scale for blocker when heading to block point
ANGLE_SEP_MIN          = math.radians(80)  # keep pursuers bracketing

# Colors
WHITE, BLUE, RED, GREEN, BLACK = (255,255,255), (50,150,255), (255,80,80), (50,200,50), (0,0,0)
PURPLE = (150, 0, 150)
ORANGE = (255, 140, 0)
LIGHT_BLUE = (200, 200, 255)
LIGHT_GRAY = (220, 220, 220)
TEAL = (0, 180, 180)

# UI / Pygame
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Pursuer-Evader - CBF + Stray + Cooperative Adversaries (Press R to Reset)")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)
small_font = pygame.font.Font(None, 24)

########################################################
# Globals for state (resettable)
########################################################
evader_x = evader_y = 0.0
p1_x = p1_y = 0.0
p2_x = p2_y = None
goal_x = goal_y = 0.0

goal_counter = 0
goal_times = []

p2_active = False
black_swan_goal_index = 1

p1_speed = PURSUER_SPEED_BASE
p2_speed = PURSUER_SPEED_BASE

prev_evader_x = prev_evader_y = 0.0
prev_p1_x = prev_p1_y = 0.0
prev_p2_x = prev_p2_y = None

running = True
game_over = False

cbf_is_active = False
black_swan_announced = False
announce_timer = 0

w_goal = 1.0
w_evade = 0.0

########################################################
# Helpers
########################################################
def clamp_speed(vx, vy, vmax):
    mag = math.hypot(vx, vy)
    if mag <= vmax or mag < CBF_EPS:
        return vx, vy
    return vx * vmax / mag, vy * vmax / mag

def spawn_far_from_evader(min_dist=120):
    for _ in range(20):
        x = random.randint(0, w)
        y = random.randint(0, h)
        if math.hypot(x - evader_x, y - evader_y) >= min_dist:
            return x, y
    return (w - evader_x, h - evader_y)

def blend_weights(dist, r_in=CBF_SAFETY_RADIUS, r_out=STRAY_RADIUS):
    """Return (w_goal, w_evade) using NEAREST pursuer distance."""
    if dist >= r_out:
        return 1.0, 0.0
    if dist <= r_in:
        return 0.0, 1.0
    t = (dist - r_in) / max(r_out - r_in, 1e-6)
    w_ev = 1.0 - t     # 1 at r_in → 0 at r_out
    w_go = 1.0 - w_ev
    return w_go, w_ev

def goal_block_point(ev, goal, frac=BLOCK_FRACTION, standoff=BLOCK_STANDOFF):
    """Point along evader->goal line where blocker should sit."""
    ex, ey = ev
    gx, gy = goal
    dx, dy = gx - ex, gy - ey
    dist = math.hypot(dx, dy)
    if dist < 1e-6:
        return gx, gy
    usable = max(dist - standoff, 0.0)
    t = min(max(frac * usable / max(dist,1e-6), 0.0), 1.0)
    bx = ex + t * dx
    by = ey + t * dy
    bx = min(max(bx, pursuer_radius), w - pursuer_radius)
    by = min(max(by, pursuer_radius), h - pursuer_radius)
    return bx, by

def angle_between(v1x, v1y, v2x, v2y):
    a = math.atan2(v1y, v1x)
    b = math.atan2(v2y, v2x)
    d = abs((a - b + math.pi) % (2*math.pi) - math.pi)
    return d

########################################################
# Multi-pursuer CBF (projection)
########################################################
def cbf_filter_multi(ev_x, ev_y, v_des, pursuers):
    """
    pursuers: list of dicts with 'x','y','vx','vy'
    Enforce ∇hᵀ (v_e - v_p) + α h >= 0 for each pursuer
    with h = ||x_e - x_p||^2 - r^2, ∇h = 2(x_e - x_p, y_e - y_p)
    """
    global cbf_is_active
    v = v_des.copy()
    intervened = False

    for pu in pursuers:
        if abs(ev_x - pu["x"]) < CBF_EPS and abs(ev_y - pu["y"]) < CBF_EPS:
            cbf_is_active = True
            return clamp_speed(1.0, 0.0, evader_speed)

    for _ in range(CBF_MAX_ITERS):
        worst_margin = float('inf')
        worst_grad = None
        worst_h = None
        worst_vp = None

        for pu in pursuers:
            rx = ev_x - pu["x"]; ry = ev_y - pu["y"]
            h = rx*rx + ry*ry - CBF_SAFETY_RADIUS**2
            grad_h = np.array([2.0*rx, 2.0*ry], dtype=float)
            v_p = np.array([pu["vx"], pu["vy"]], dtype=float)
            margin = float(grad_h @ (v - v_p) + CBF_ALPHA * h)
            if margin < worst_margin:
                worst_margin = margin
                worst_grad = grad_h
                worst_h = h
                worst_vp = v_p

        if worst_margin >= 0.0:
            break

        denom = float(worst_grad @ worst_grad)
        if denom < CBF_EPS:
            continue
        hdot = float(worst_grad @ (v - worst_vp))
        lam = (-(hdot + CBF_ALPHA * worst_h)) / denom
        v = v + lam * worst_grad
        intervened = True

    cbf_is_active = intervened
    vx, vy = clamp_speed(v[0], v[1], evader_speed)
    return vx, vy

########################################################
# Resettable state
########################################################
def reset_sim():
    """Reset the entire simulation state (bound to hotkey R)."""
    global evader_x, evader_y, p1_x, p1_y, p2_x, p2_y
    global goal_x, goal_y, goal_counter, goal_times
    global p2_active, black_swan_goal_index
    global p1_speed, p2_speed
    global prev_evader_x, prev_evader_y, prev_p1_x, prev_p1_y, prev_p2_x, prev_p2_y
    global running, game_over, cbf_is_active, black_swan_announced, announce_timer
    global w_goal, w_evade

    evader_x =  w // 4
    evader_y =  h // 2
    p1_x = 3 * w // 4
    p1_y = h // 2
    p2_x, p2_y = None, None

    goal_x, goal_y = random.randint(0, w), random.randint(0, h)

    goal_counter = 0
    goal_times = []

    p2_active = False
    black_swan_goal_index = random.randint(1, max(1, max_goals - 1))

    p1_speed = PURSUER_SPEED_BASE
    p2_speed = PURSUER_SPEED_BASE

    prev_evader_x, prev_evader_y = evader_x, evader_y
    prev_p1_x, prev_p1_y = p1_x, p1_y
    prev_p2_x, prev_p2_y = None, None

    running = True
    game_over = False

    cbf_is_active = False
    black_swan_announced = False
    announce_timer = 0

    w_goal = 1.0
    w_evade = 0.0

# Initialize once
reset_sim()

########################################################
# Main
########################################################
while running:
    clock.tick(60)
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                running = False
            if event.key == pygame.K_r:
                reset_sim()   # <-- HOTKEY: reset

    if not game_over:
        # Estimate pursuer velocities
        p1_vx_est = p1_x - prev_p1_x
        p1_vy_est = p1_y - prev_p1_y
        if p2_active and prev_p2_x is not None:
            p2_vx_est = p2_x - prev_p2_x
            p2_vy_est = p2_y - prev_p2_y
        else:
            p2_vx_est = 0.0
            p2_vy_est = 0.0

        # Goal vector
        gdx = goal_x - evader_x
        gdy = goal_y - evader_y
        gdist = math.hypot(gdx, gdy)
        v_goal = np.array([0.0, 0.0])
        if gdist > 0:
            v_goal = np.array([gdx / gdist, gdy / gdist]) * evader_speed

        # Evasion dir: sum push-aways, weighted by inverse distance
        evade_vec = np.array([0.0, 0.0])
        dists = []
        # p1
        r1x = evader_x - p1_x; r1y = evader_y - p1_y
        d1 = math.hypot(r1x, r1y)
        if d1 > 0:
            evade_vec += np.array([r1x/d1, r1y/d1]) * (1.0 / d1)
            dists.append(d1)
        # p2
        if p2_active:
            r2x = evader_x - p2_x; r2y = evader_y - p2_y
        else:
            r2x = r2y = 0.0
        if p2_active:
            d2 = math.hypot(r2x, r2y)
            if d2 > 0:
                evade_vec += np.array([r2x/d2, r2y/d2]) * (1.0 / d2)
                dists.append(d2)

        if np.linalg.norm(evade_vec) > 0:
            evade_vec = (evade_vec / np.linalg.norm(evade_vec)) * evader_speed

        # Blend by nearest pursuer distance
        nearest = min(dists) if dists else float('inf')
        w_goal, w_evade = blend_weights(nearest)

        # Desired velocity (blend)
        v_des = v_goal * w_goal + evade_vec * w_evade
        n = np.linalg.norm(v_des)
        if n > 0:
            v_des = (v_des / n) * evader_speed

        # CBF against both pursuers
        pursuers = [{"x": p1_x, "y": p1_y, "vx": p1_vx_est, "vy": p1_vy_est}]
        if p2_active:
            pursuers.append({"x": p2_x, "y": p2_y, "vx": p2_vx_est, "vy": p2_vy_est})

        safe_vx, safe_vy = cbf_filter_multi(evader_x, evader_y, v_des, pursuers)

        # Update evader
        prev_evader_x, prev_evader_y = evader_x, evader_y
        evader_x += safe_vx
        evader_y += safe_vy
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # =========================
        # Cooperative Pursuers
        # =========================
        # 1) P1 = Chaser (classic pursuit)
        dx1 = evader_x - p1_x; dy1 = evader_y - p1_y
        d1 = math.hypot(dx1, dy1)
        if d1 > 0:
            step1x = (dx1 / d1) * p1_speed
            step1y = (dy1 / d1) * p1_speed
        else:
            step1x = step1y = 0.0

        prev_p1_x, prev_p1_y = p1_x, p1_y
        p1_x += step1x; p1_y += step1y
        p1_x = max(pursuer_radius, min(w - pursuer_radius, p1_x))
        p1_y = max(pursuer_radius, min(h - pursuer_radius, p1_y))

        # 2) P2 = Blocker/Interceptor (after spawn)
        if p2_active:
            bx, by = goal_block_point((evader_x, evader_y), (goal_x, goal_y),
                                      frac=BLOCK_FRACTION, standoff=BLOCK_STANDOFF)

            # keep angular separation (bracketing)
            v1x, v1y = p1_x - evader_x, p1_y - evader_y
            v2x, v2y = bx - evader_x, by - evader_y
            ang = angle_between(v1x, v1y, v2x, v2y)
            if ang < ANGLE_SEP_MIN:
                sgn = -1.0 if (v1x * (-v2y) + v1y * v2x) > 0 else 1.0
                rot90x = -v2y * sgn
                rot90y =  v2x * sgn
                m = math.hypot(rot90x, rot90y)
                if m > 1e-6:
                    rot90x /= m; rot90y /= m
                    offset = 60.0
                    bx += rot90x * offset
                    by += rot90y * offset
                    bx = min(max(bx, pursuer_radius), w - pursuer_radius)
                    by = min(max(by, pursuer_radius), h - pursuer_radius)

            dx2 = bx - p2_x; dy2 = by - p2_y
            d2p = math.hypot(dx2, dy2)
            if d2p > 0:
                step2x = (dx2 / d2p) * p2_speed * BLOCK_REPOSITION_GAIN
                step2y = (dy2 / d2p) * p2_speed * BLOCK_REPOSITION_GAIN
            else:
                step2x = step2y = 0.0

            prev_p2_x, prev_p2_y = p2_x, p2_y
            p2_x += step2x; p2_y += step2y
            p2_x = max(pursuer_radius, min(w - pursuer_radius, p2_x))
            p2_y = max(pursuer_radius, min(h - pursuer_radius, p2_y))

        # Capture checks (either pursuer)
        if math.hypot(evader_x - p1_x, evader_y - p1_y) <= COLLISION_DISTANCE:
            game_over = True
        if p2_active and math.hypot(evader_x - p2_x, evader_y - p2_y) <= COLLISION_DISTANCE:
            game_over = True
        if game_over:
            # stop movement, but let R reset immediately
            pass

        # Goal collection
        final_goal_dist = math.hypot(evader_x - goal_x, evader_y - goal_y)
        if (not game_over) and goal_counter < max_goals and final_goal_dist <= evader_radius + goal_radius:
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time)
            goal_counter += 1

            # Black swan trigger: spawn second pursuer and boost speeds
            if (not p2_active) and (goal_counter == black_swan_goal_index):
                p2_active = True
                p2_x, p2_y = spawn_far_from_evader(min_dist=CBF_SAFETY_RADIUS * 1.5)
                prev_p2_x, prev_p2_y = p2_x, p2_y
                black_swan_announced = True
                announce_timer = 120  # ~2 seconds

                # Coop speed boost
                p1_speed = PURSUER_SPEED_COOP
                p2_speed = PURSUER_SPEED_COOP

            if goal_counter < max_goals:
                goal_x, goal_y = random.randint(0, w), random.randint(0, h)
                goal_spawn_time = pygame.time.get_ticks() / 1000.0
            else:
                game_over = True

    ########################################################
    # Draw
    ########################################################
    # Safety + stray circles for each pursuer
    pygame.draw.circle(screen, LIGHT_BLUE, (int(p1_x), int(p1_y)), int(CBF_SAFETY_RADIUS), 2)
    pygame.draw.circle(screen, LIGHT_GRAY, (int(p1_x), int(p1_y)), int(STRAY_RADIUS), 1)
    if p2_active:
        pygame.draw.circle(screen, LIGHT_BLUE, (int(p2_x), int(p2_y)), int(CBF_SAFETY_RADIUS), 2)
        pygame.draw.circle(screen, LIGHT_GRAY, (int(p2_x), int(p2_y)), int(STRAY_RADIUS), 1)

    # Agents + lines
    pygame.draw.circle(screen, RED, (int(p1_x), int(p1_y)), pursuer_radius)
    if p2_active:
        pygame.draw.circle(screen, ORANGE, (int(p2_x), int(p2_y)), pursuer_radius)
        bx, by = goal_block_point((evader_x, evader_y), (goal_x, goal_y),
                                  frac=BLOCK_FRACTION, standoff=BLOCK_STANDOFF)
        pygame.draw.circle(screen, TEAL, (int(bx), int(by)), 5)
        pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(p2_x), int(p2_y)), 1)
        pygame.draw.line(screen, TEAL, (int(p2_x), int(p2_y)), (int(bx), int(by)), 1)

    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(p1_x), int(p1_y)), 1)

    # Goal
    if goal_counter < max_goals:
        pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)

    # HUD
    screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 10))
    mode = "CBF ACTIVE" if cbf_is_active else ("STRAYING" if w_evade > 0 else "GOAL-SEEKING")
    mode_color = RED if cbf_is_active else (ORANGE if w_evade > 0 else GREEN)
    screen.blit(small_font.render(f"Mode: {mode}", True, mode_color), (10, 44))
    screen.blit(small_font.render("Adversaries: Cooperative (Chaser + Blocker)", True, PURPLE), (10, 64))
    screen.blit(small_font.render(f"W_Goal: {w_goal:.2f}", True, BLACK), (10, 86))
    screen.blit(small_font.render(f"W_Evade: {w_evade:.2f}", True, BLACK), (10, 106))
    screen.blit(small_font.render("Hotkeys: Q quit | R reset", True, BLACK), (10, 128))

    if black_swan_announced and announce_timer > 0:
        msg = font.render("BLACK SWAN: Second pursuer joined!", True, ORANGE)
        screen.blit(msg, msg.get_rect(center=(w//2, 40)))
        announce_timer -= 1
        if announce_timer == 0:
            black_swan_announced = False

    if game_over:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals collected!", True, GREEN)
        else:
            msg = font.render("GAME OVER - Caught!", True, RED)
        screen.blit(msg, msg.get_rect(center=(w//2, h//2)))
        # (R) works even while game_over

    pygame.display.flip()

pygame.quit()
